In [1]:
import mlrun
project_name = "jw-proj"
project = mlrun.get_or_create_project(project_name, "./")

> 2024-11-04 11:07:02,159 [info] Project loaded successfully: {"project_name":"jw-proj"}


# Dask

In [14]:
dask_cluster_name = "dask-cluster"
dask_cluster = mlrun.new_function(
    dask_cluster_name, kind="dask", image="mlrun/mlrun"
)

dask_cluster.apply(mlrun.mount_v3io())
project.set_function(dask_cluster)

In [15]:
# set range for # of replicas with replicas and max_replicas
dask_cluster.spec.min_replicas = 1
dask_cluster.spec.max_replicas = 4

# set the use of dask remote cluster (distributed)
dask_cluster.spec.remote = True
dask_cluster.spec.service_type = "NodePort"

# set dask memory and cpu limits
dask_cluster.with_worker_requests(mem="2G", cpu="2")

In [16]:
def inc(x):
	return x+2

In [17]:
def hndlr(context, x=1, y=2):
	context.logger.info('params: x={},y={}'.format(x, y))
	x = context.dask_client.submit(inc, x)
	print(x.result())
	context.log_result('dask-try', x.result())

In [18]:
myrun = dask_cluster.run(handler=hndlr, name='dask-try', params={'x': 12, 'y' : 3})

> 2024-11-01 15:36:05,726 [info] Storing function: {"db":"http://mlrun-api:8080","name":"dask-try","uid":"c57770ef2d1a4244a53d4f085557b741"}
> 2024-11-01 15:37:22,881 [info] Trying dask client at: tcp://mlrun-dask-cluster-0e3a2551-8.default-tenant:8786
> 2024-11-01 15:37:22,944 [info] Using remote dask scheduler (mlrun-dask-cluster-0e3a2551-8) at: tcp://mlrun-dask-cluster-0e3a2551-8.default-tenant:8786


Mismatched versions found

+---------+----------------+----------------+---------+
| Package | Client         | Scheduler      | Workers |
+---------+----------------+----------------+---------+
| python  | 3.9.18.final.0 | 3.9.20.final.0 | None    |
| toolz   | 0.12.1         | 0.12.0         | None    |
+---------+----------------+----------------+---------+


> 2024-11-01 15:37:22,970 [info] params: x=12,y=3
14


project,uid,iter,start,state,kind,name,labels,inputs,parameters,results
jw-proj,...5557b741,0,Nov 01 15:36:05,completed,run,dask-try,v3io_user=normal-userkind=daskowner=normal-userhost=jupyter-b4d4cbbb8-zpcwv,,x=12y=3,dask-try=14


> 2024-11-01 15:37:33,010 [info] Run execution finished: {"name":"dask-try","status":"completed"}


# Local

In [19]:
%%writefile data-prep.py

import pandas as pd
from sklearn.datasets import load_breast_cancer

import mlrun


@mlrun.handler(outputs=["dataset", "label_column"])
def breast_cancer_generator():
    """
    A function which generates the breast cancer dataset
    """
    breast_cancer = load_breast_cancer()
    breast_cancer_dataset = pd.DataFrame(
        data=breast_cancer.data, columns=breast_cancer.feature_names
    )
    breast_cancer_labels = pd.DataFrame(data=breast_cancer.target, columns=["label"])
    breast_cancer_dataset = pd.concat(
        [breast_cancer_dataset, breast_cancer_labels], axis=1
    )

    return breast_cancer_dataset, "label"

Writing data-prep.py


In [20]:
data_gen_fn = project.set_function(
    "data-prep.py",
    name="data-prep",
    kind="job",
    image="mlrun/mlrun",
    handler="breast_cancer_generator",
)

In [21]:
gen_data_run = project.run_function("data-prep", local=True)

> 2024-11-01 15:37:46,911 [info] Storing function: {"db":"http://mlrun-api:8080","name":"data-prep-breast-cancer-generator","uid":"79b067472ba44ffa84aead4a50eabd47"}


project,uid,iter,start,state,kind,name,labels,inputs,parameters,results,artifacts
jw-proj,...50eabd47,0,Nov 01 15:37:46,completed,run,data-prep-breast-cancer-generator,v3io_user=normal-userkind=localowner=normal-userhost=jupyter-b4d4cbbb8-zpcwv,,,label_column=label,dataset


> 2024-11-01 15:37:56,101 [info] Run execution finished: {"name":"data-prep-breast-cancer-generator","status":"completed"}


# Job

In [22]:
# Import the function
trainer = mlrun.import_function("hub://auto_trainer")

In [23]:
trainer_run = project.run_function(
    trainer,
    inputs={"dataset": gen_data_run.outputs["dataset"]},
    params={
        "model_class": "sklearn.ensemble.RandomForestClassifier",
        "train_test_split_size": 0.2,
        "label_columns": "label",
        "model_name": "cancer",
    },
    handler="train",
)

> 2024-11-01 15:38:27,537 [info] Storing function: {"db":"http://mlrun-api:8080","name":"auto-trainer-train","uid":"3971ce02537d448f9904c6ab1df54d6f"}
> 2024-11-01 15:38:27,895 [info] Job is running in the background, pod: auto-trainer-train-c8mzz
> 2024-11-01 15:38:31,902 [info] Sample set not given, using the whole training set as the sample set
> 2024-11-01 15:38:32,144 [info] training 'cancer'
> 2024-11-01 15:38:34,754 [info] To track results use the CLI: {"info_cmd":"mlrun get run 3971ce02537d448f9904c6ab1df54d6f -p jw-proj","logs_cmd":"mlrun logs 3971ce02537d448f9904c6ab1df54d6f -p jw-proj"}
> 2024-11-01 15:38:34,755 [info] Or click for UI: {"ui_url":"https://dashboard.default-tenant.app.vmdev54.lab.iguazeng.com/mlprojects/jw-proj/jobs/monitor/3971ce02537d448f9904c6ab1df54d6f/overview"}
> 2024-11-01 15:38:34,755 [info] Run execution finished: {"name":"auto-trainer-train","status":"completed"}


project,uid,iter,start,state,kind,name,labels,inputs,parameters,results,artifacts
jw-proj,...1df54d6f,0,Nov 01 15:38:31,completed,run,auto-trainer-train,v3io_user=normal-userkind=jobowner=normal-usermlrun/client_version=1.7.0-rc52mlrun/client_python_version=3.9.18host=auto-trainer-train-c8mzz,dataset,model_class=sklearn.ensemble.RandomForestClassifiertrain_test_split_size=0.2label_columns=labelmodel_name=cancer,accuracy=0.9912280701754386f1_score=0.9928057553956835precision_score=0.9857142857142858recall_score=1.0,feature-importancetest_setconfusion-matrixroc-curvescalibration-curvemodel


> 2024-11-01 15:38:39,110 [info] Run execution finished: {"name":"auto-trainer-train","status":"completed"}


# Serving

In [24]:
serving_fn = mlrun.new_function("serving", image="mlrun/mlrun", kind="serving")
serving_fn.add_model(
    "cancer-classifier",
    model_path=trainer_run.outputs["model"],
    class_name="mlrun.frameworks.sklearn.SklearnModelServer",
)

In [25]:
project.deploy_function(serving_fn)

> 2024-11-01 15:38:45,137 [info] Starting remote function deploy
2024-11-01 15:38:45  (info) Deploying function
2024-11-01 15:38:45  (info) Building
2024-11-01 15:38:45  (info) Staging files and preparing base images
2024-11-01 15:38:45  (warn) Using user provided base image, runtime interpreter version is provided by the base image
2024-11-01 15:38:45  (info) Building processor image
2024-11-01 15:40:40  (info) Build complete
2024-11-01 15:41:02  (info) Function deploy complete
> 2024-11-01 15:41:09,573 [info] Successfully deployed function: {"external_invocation_urls":["jw-proj-serving.default-tenant.app.vmdev54.lab.iguazeng.com/"],"internal_invocation_urls":["nuclio-jw-proj-serving.default-tenant.svc.cluster.local:8080"]}


DeployStatus(state=ready, outputs={'endpoint': 'http://jw-proj-serving.default-tenant.app.vmdev54.lab.iguazeng.com/', 'name': 'jw-proj-serving'})

In [29]:
sample = {
    "inputs": [
        [
            1.371e01,
            2.083e01,
            9.020e01,
            5.779e02,
            1.189e-01,
            1.645e-01,
            9.366e-02,
            5.985e-02,
            2.196e-01,
            7.451e-02,
            5.835e-01,
            1.377e00,
            3.856e00,
            5.096e01,
            8.805e-03,
            3.029e-02,
            2.488e-02,
            1.448e-02,
            1.486e-02,
            5.412e-03,
            1.706e01,
            2.814e01,
            1.106e02,
            8.970e02,
            1.654e-01,
            3.682e-01,
            2.678e-01,
            1.556e-01,
            3.196e-01,
            1.151e-01,
        ]
    ]
}

In [30]:
serving_fn.invoke(path=f"/v2/models/cancer-classifier/infer", body=sample)

> 2024-11-01 15:42:35,483 [info] Invoking function: {"method":"POST","path":"http://nuclio-jw-proj-serving.default-tenant.svc.cluster.local:8080/v2/models/cancer-classifier/infer"}


{'id': '13c820d8-29d9-4a21-90bc-ab90c384df33',
 'model_name': 'cancer-classifier',
 'outputs': [0],
 'timestamp': '2024-11-01 15:42:35.493693+00:00',
 'model_version': 'latest'}

# Handler

In [72]:
import pickle
from mlrun import new_task

!pip install xgboost
from xgboost import XGBClassifier

from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

In [73]:
def handler(context):
    context.set_label('category', 'tests')
    print('artifact_path={}'.format(context.artifact_path))
    dataset = load_iris(as_frame=True)
    clf = XGBClassifier(n_estimators=2, max_depth=2, learning_rate=1, objective= "binary:logistic")
    X,y = dataset.data, dataset.target
    X_train, x_test, y_train, y_test = train_test_split(X, y, train_size=0.8, test_size=0.2, random_state=0)
    clf.fit(X_train, y_train)
    context.log_model("clf_model", body=pickle.dumps(clf), model_file="clf.pkl", framework="xgboost", label_column='label', tag="fgvndgt1")
    context.log_dataset("ds",df=x_test, tag="fgvndgt")

In [ ]:
task = new_task(name='kmqrijzfki', handler=handler, project=project_name, **{})
func = mlrun.new_function(name='func', kind='job', image='mlrun/mlrun')
func.save()
run_object = mlrun.run_function(func, local=True, base_task=task)

> 2024-11-01 16:06:26,803 [warning] run command, file or code were not specified
> 2024-11-01 16:06:26,805 [info] Storing function: {"db":null,"name":"kmqrijzfki","uid":"0892335d65734b7a9d1c3fde8fefed9e"}
artifact_path=v3io:///projects/jw-proj/artifacts/0892335d65734b7a9d1c3fde8fefed9e


project,uid,iter,start,state,kind,name,labels,inputs,parameters,results,artifacts
jw-proj,...8fefed9e,0,Nov 01 16:06:26,completed,run,kmqrijzfki,v3io_user=normal-userkind=handlerowner=normal-userhost=jupyter-b4d4cbbb8-zpcwvcategory=tests,,,,clf_modelds


> 2024-11-01 16:06:27,626 [info] Run execution finished: {"name":"kmqrijzfki","status":"completed"}


# Nuclio

In [40]:
%%writefile nuclio-func.py

import socket
import pandas as pd


def hyper_func2(context, data, p1, p2, p3):
    print(data.as_df().head())
    context.logger.info(f"p2={p2}, p3={p3}, r1={p2 * p3} at {socket.gethostname()}")
    context.log_result("r1", p2 * p3)
    raw_data = {
        "first_name": ["Jason", "Molly", "Tina", "Jake", "Amy"],
        "age": [42, 52, 36, 24, 73],
        "testScore": [25, 94, 57, 62, 70],
    }
    df = pd.DataFrame(raw_data, columns=["first_name", "age", "testScore"])
    context.log_dataset("mydf", df=df, stats=True)

Overwriting nuclio-func.py


In [41]:
fn = mlrun.code_to_function(filename="nuclio-func.py", handler="hyper_func2", name="hyper-tst2", kind="nuclio:mlrun", image="mlrun/mlrun")
# replicas * workers need to match or exceed parallel_runs
fn.spec.replicas = 2
fn.with_http(workers=2)
fn.deploy()

> 2024-11-01 15:49:05,776 [warning] Adding HTTP trigger despite the default HTTP trigger creation being disabled
> 2024-11-01 15:49:05,781 [info] Starting remote function deploy
2024-11-01 15:49:06  (info) Deploying function
2024-11-01 15:49:06  (info) Building
2024-11-01 15:49:06  (info) Staging files and preparing base images
2024-11-01 15:49:06  (warn) Using user provided base image, runtime interpreter version is provided by the base image
2024-11-01 15:49:06  (info) Building processor image
2024-11-01 15:50:02  (info) Build complete
2024-11-01 15:51:02  (info) Function deploy complete
> 2024-11-01 15:51:08,145 [info] Successfully deployed function: {"external_invocation_urls":["jw-proj-hyper-tst2.default-tenant.app.vmdev54.lab.iguazeng.com/"],"internal_invocation_urls":["nuclio-jw-proj-hyper-tst2.default-tenant.svc.cluster.local:8080"]}


'http://jw-proj-hyper-tst2.default-tenant.app.vmdev54.lab.iguazeng.com/'

In [42]:
import nest_asyncio

nest_asyncio.apply()

In [43]:
grid_params = {"p2": [2, 1, 4, 1], "p3": [10, 20]}
task = mlrun.new_task(
    params={"p1": 8},
    inputs={"data": "https://s3.wasabisys.com/iguazio/data/iris/iris_dataset.csv"},
)
task.with_hyper_params(
    grid_params, selector="r1", strategy="grid", parallel_runs=4, max_errors=3
)
run = fn.run(task)

> 2024-11-01 15:51:12,433 [info] Storing function: {"db":"http://mlrun-api:8080","name":"hyper-tst2-hyper-func2","uid":"632c4599d5984a57afb359a9ea201c32"}
> --------------- Iteration: (1) ---------------
2024-11-01 15:51:12.799663  info   logging run results to: http://mlrun-api:8080  worker_id=0
2024-11-01 15:51:15.676397  info   p2=2, p3=10, r1=20 at nuclio-jw-proj-hyper-tst2-6f48476bd4-gn4zc  worker_id=0

> --------------- Iteration: (2) ---------------
2024-11-01 15:51:12.800199  info   logging run results to: http://mlrun-api:8080  worker_id=0
2024-11-01 15:51:15.580803  info   p2=1, p3=10, r1=10 at nuclio-jw-proj-hyper-tst2-6f48476bd4-dfvcp  worker_id=0

> --------------- Iteration: (3) ---------------
2024-11-01 15:51:12.800168  info   logging run results to: http://mlrun-api:8080  worker_id=1
2024-11-01 15:51:15.793398  info   p2=4, p3=10, r1=40 at nuclio-jw-proj-hyper-tst2-6f48476bd4-gn4zc  worker_id=1

> --------------- Iteration: (4) ---------------
2024-11-01 15:51:15.75849

project,uid,iter,start,state,kind,name,labels,inputs,parameters,results,artifacts
jw-proj,...ea201c32,0,Nov 01 15:51:12,completed,run,hyper-tst2-hyper-func2,v3io_user=normal-userkind=remoteowner=normal-user,data,p1=8,best_iteration=7r1=80,mydfiteration_resultsparallel_coordinates


> 2024-11-01 15:51:19,336 [info] Run execution finished: {"name":"hyper-tst2-hyper-func2","status":"completed"}


# Mpi

In [75]:
%%writefile mpi-func.py

import time

def do_nothing():
	time.sleep(0)
	return "nevermind"

Overwriting mpi-func.py


In [76]:
mpijob = mlrun.code_to_function(name="mpijob-func", filename="mpi-func.py", kind="mpijob", image="mlrun/mlrun", handler="do_nothing")

In [77]:
mpijob.spec.replicas = 3

In [78]:
mpijob.run()

> 2024-11-01 16:07:25,052 [info] Storing function: {"db":"http://mlrun-api:8080","name":"mpijob-func-do-nothing","uid":"36139c08bd654d2c9e08b6ba3c3164ad"}
> 2024-11-01 16:07:31,450 [info] MpiJob mpijob-func-do-nothing-3d295a33 launcher pod mpijob-func-do-nothing-3d295a33-launcher state running
+ POD_NAME=mpijob-func-do-nothing-3d295a33-worker-0
+ shift
+ /opt/kube/kubectl exec mpijob-func-do-nothing-3d295a33-worker-0 -- /bin/sh -c  orted -mca ess "env" -mca ess_base_jobid "3369730048" -mca ess_base_vpid 1 -mca ess_base_num_procs "4" -mca orte_node_regex "mpijob-func-do-nothing-[1:3]d295a33-launcher,mpijob-func-do-nothing-[1:3]d295a33-worker-0,mpijob-func-do-nothing-[1:3]d295a33-worker-1,mpijob-func-do-nothing-[1:3]d295a33-worker-2@0(4)" -mca orte_hnp_uri "3369730048.0;tcp://10.233.116.46:59563" -mca plm "rsh" --tree-spawn -mca routed "radix" -mca orte_parent_uri "3369730048.0;tcp://10.233.116.46:59563" -mca plm_rsh_agent "/etc/mpi/kubexec.sh" -mca orte_default_hostfile "/etc/mpi/hostfi

project,uid,iter,start,state,kind,name,labels,inputs,parameters,results
jw-proj,...3c3164ad,0,Nov 01 16:07:34,completed,run,mpijob-func-do-nothing,v3io_user=normal-userkind=mpijobowner=normal-usermlrun/client_version=1.7.0-rc52mlrun/client_python_version=3.9.18mlrun/job=mpijob-func-do-nothing-3d295a33host=mpijob-func-do-nothing-3d295a33-worker-0,,,return=nevermind


> 2024-11-01 16:08:05,828 [info] Run execution finished: {"name":"mpijob-func-do-nothing","status":"completed"}


# Application

In [79]:
# Specify pre built vizro image
application = project.set_function(name="vizro", kind="application", image="kharchukt/vizro")
application.set_internal_application_port(8050)
# Deploy to nuclio
application.deploy()

> 2024-11-01 16:09:03,220 [info] Starting remote function deploy
2024-11-01 16:09:03  (info) Deploying function
2024-11-01 16:09:03  (info) Building
2024-11-01 16:09:03  (info) Staging files and preparing base images
2024-11-01 16:09:03  (info) Building processor image
2024-11-01 16:10:18  (info) Build complete
2024-11-01 16:11:15  (info) Function deploy complete
> 2024-11-01 16:11:17,188 [info] Successfully deployed function.
> 2024-11-01 16:11:24,448 [info] Successfully created API gateway: {"url":"https://jw-proj-vizro-latest.default-tenant.app.vmdev54.lab.iguazeng.com"}


'https://jw-proj-vizro-latest.default-tenant.app.vmdev54.lab.iguazeng.com'

In [80]:
# Test invocation
application.invoke("/", verify=False)

/conda/envs/mlrun-base/lib/python3.9/site-packages/urllib3/connectionpool.py:1064: InsecureRequestWarning:

Unverified HTTPS request is being made to host 'jw-proj-vizro-latest.default-tenant.app.vmdev54.lab.iguazeng.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings



<Response [200]>

# Spark

In [2]:
def my_spark(context=None):
	from time import sleep
	sleep(0)

In [5]:
from mlrun.run import new_function
function_pbvtwxlwcd = new_function(kind='spark', command='/v3io/bigdata/naipi_files/pyspark_basic_tests.py', name='tqrcj')
if None:
	function_pbvtwxlwcd.set_state_thresholds(None)
function_pbvtwxlwcd.metadata.tag = ""

function_pbvtwxlwcd.with_executor_limits(cpu="2200m")
function_pbvtwxlwcd.with_driver_limits(cpu="2500m")
project.set_function(function_pbvtwxlwcd)
function_pbvtwxlwcd.with_executor_requests(mem='1G', cpu='0.1')
function_pbvtwxlwcd.with_driver_requests(mem='1G', cpu='0.1')
function_pbvtwxlwcd.spec.replicas = 2
function_pbvtwxlwcd.with_igz_spark()
function_pbvtwxlwcd.spec.args = ['--json-path=/v3io/bigdata/imdb_movies.json']
function_pbvtwxlwcd.spec.deps['jars'] += ['local:///spark/3rd_party/mysql-connector-java-8.0.13.jar']
sr = function_pbvtwxlwcd.run(artifact_path='/User/artifacts', watch=False, notifications=[])

> 2024-11-04 11:07:57,126 [info] Storing function: {"db":null,"name":"tqrcj","uid":"c5d926ef593945e8a3b474d3146af9c4"}


project,uid,iter,start,state,kind,name,labels,inputs,parameters,results
jw-proj,...146af9c4,0,Nov 04 11:07:57,running,,tqrcj,v3io_user=normal-userkind=sparkowner=normal-usermlrun/client_version=1.7.0-rc52mlrun/client_python_version=3.9.18,,,


> 2024-11-04 11:07:58,091 [info] Run execution finished: {"name":"tqrcj","status":"running"}


# Databriks

In [81]:
import os
from mlrun.runtimes.function_reference import FunctionReference

In [82]:
# If using a Databricks data store, for example, set the credentials:
os.environ["DATABRICKS_HOST"] = "DATABRICKS_HOST"
os.environ["DATABRICKS_TOKEN"] = "DATABRICKS_TOKEN"
os.environ["DATABRICKS_CLUSTER_ID"] = "DATABRICKS_CLUSTER_ID"

In [83]:
def add_databricks_env(function):
    job_env = {
        "DATABRICKS_HOST": os.environ["DATABRICKS_HOST"],
        "DATABRICKS_CLUSTER_ID": os.environ.get("DATABRICKS_CLUSTER_ID"),
    }

    for name, val in job_env.items():
        function.spec.env.append({"name": name, "value": val})

In [89]:
secrets = {"DATABRICKS_TOKEN": os.environ["DATABRICKS_TOKEN"]}

project.set_secrets(secrets)

code = """
def print_kwargs(**kwargs):
    print(f"kwargs: {kwargs}")
"""

function_ref = FunctionReference(
    kind="databricks",
    code=code,
    image="mlrun/mlrun",
    name="databricks-function",
)

function = function_ref.to_function()

add_databricks_env(function=function)

run = function.run(
    handler="print_kwargs",
    project=project_name,
    params={
        "param1": "value1",
        "param2": "value2",
        # "task_parameters": {"timeout_minutes": 1},
    },
)

> 2024-11-01 16:26:25,963 [info] Storing function: {"db":"http://mlrun-api:8080","name":"databricks-function-print-kwargs","uid":"153fa25d6565400a9a53254a5e283bb2"}
> 2024-11-01 16:26:26,261 [info] Job is running in the background, pod: databricks-function-print-kwargs-687pq
> 2024-11-01 16:31:32,626 [error] Execution error, Traceback (most recent call last):
  File "/opt/conda/lib/python3.9/site-packages/urllib3/connection.py", line 174, in _new_conn
    conn = connection.create_connection(
  File "/opt/conda/lib/python3.9/site-packages/urllib3/util/connection.py", line 72, in create_connection
    for res in socket.getaddrinfo(host, port, family, socket.SOCK_STREAM):
  File "/opt/conda/lib/python3.9/socket.py", line 966, in getaddrinfo
    for res in _socket.getaddrinfo(host, port, family, type, proto, flags):
socket.gaierror: [Errno -3] Temporary failure in name resolution

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File

> 2024-11-01 16:31:44,591 [info] Run execution finished: {"name":"databricks-function-print-kwargs","status":"error"}
> 2024-11-01 16:31:44,592 [error] Run did not finish successfully: {"state":"error","status":{"error":"Timed out after 0:05:00, caused by: HTTPSConnectionPool(host='databricks_host', port=443): Max retries exceeded with url: /api/2.0/preview/scim/v2/Me (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7f80b336feb0>: Failed to establish a new connection: [Errno -3] Temporary failure in name resolution'))","last_update":"2024-11-01T16:31:32.717315+00:00","start_time":"2024-11-01T16:26:29.029244+00:00","state":"error"}}


RunError: Timed out after 0:05:00, caused by: HTTPSConnectionPool(host='databricks_host', port=443): Max retries exceeded with url: /api/2.0/preview/scim/v2/Me (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7f80b336feb0>: Failed to establish a new connection: [Errno -3] Temporary failure in name resolution'))